<a href="https://colab.research.google.com/github/cmaurini/MU4MES03/blob/master/colab-poisson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
arch = os.getenv("ARGS", "real")

In [2]:
try:
    import google.colab  # noqa: F401
except ImportError:
    import ufl
    import dolfinx
else:
    try:
        import ufl
        import dolfinx
    except ImportError:
        if arch != "complex":
            !wget "https://fem-on-colab.github.io/releases/fenicsx-install-real.sh" -O "/tmp/fenicsx-install.sh" && bash "/tmp/fenicsx-install.sh"
        else:
            !wget "https://fem-on-colab.github.io/releases/fenicsx-install-complex.sh" -O "/tmp/fenicsx-install.sh" && bash "/tmp/fenicsx-install.sh"
        import ufl
        import dolfinx

--2023-09-29 04:56:16--  https://fem-on-colab.github.io/releases/fenicsx-install-real.sh
Resolving fem-on-colab.github.io (fem-on-colab.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to fem-on-colab.github.io (fem-on-colab.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4319 (4.2K) [application/x-sh]
Saving to: ‘/tmp/fenicsx-install.sh’

/tmp/fenicsx-instal 100%[===================>]   4.22K  --.-KB/s    in 0s      

2023-09-29 04:56:16 (49.7 MB/s) - ‘/tmp/fenicsx-install.sh’ saved [4319/4319]

+ INSTALL_PREFIX=/usr/local
++ echo /usr/local
++ awk -F/ '{print NF-1}'
+ INSTALL_PREFIX_DEPTH=2
+ PROJECT_NAME=fem-on-colab
+ SHARE_PREFIX=/usr/local/share/fem-on-colab
+ FENICSX_INSTALLED=/usr/local/share/fem-on-colab/fenicsx.installed
+ [[ ! -f /usr/local/share/fem-on-colab/fenicsx.installed ]]
+ PYBIND11_INSTALL_SCRIPT_PATH=https://github.com/fem-on-colab/fem-on-colab.github.io/raw/4e2d907/releases/pybind1

In [3]:
import numpy as np
import petsc4py
import petsc4py.PETSc
if arch != "complex":
    assert not np.issubdtype(petsc4py.PETSc.ScalarType, np.complexfloating)
else:
    assert np.issubdtype(petsc4py.PETSc.ScalarType, np.complexfloating)

In [4]:
import dolfinx.fem
import dolfinx.fem.petsc
import dolfinx.mesh
import dolfinx.io

In [5]:
import mpi4py
import mpi4py.MPI

In [6]:
mesh = dolfinx.mesh.create_unit_interval(mpi4py.MPI.COMM_WORLD, 3)

In [7]:
V = dolfinx.fem.functionspace(mesh, ("CG", 1))

In [8]:
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

In [9]:
dx = ufl.dx

In [10]:
f = dolfinx.fem.Function(V)
dim_V = f.vector.local_size
assert dim_V == 4
f.vector[:] = np.arange(1, dim_V + 1)

In [11]:
a = ufl.inner(u, v) * dx
F = ufl.inner(f, v) * dx
a_cpp = dolfinx.fem.form(a)
F_cpp = dolfinx.fem.form(F)

In [12]:
A = dolfinx.fem.petsc.assemble_matrix(a_cpp)
b = dolfinx.fem.petsc.assemble_vector(F_cpp)

In [13]:
A.assemble()

In [14]:
solution = dolfinx.fem.Function(V)

In [15]:
for (package_number, package) in enumerate((None, "mumps", "superlu", "superlu_dist")):
    ksp = petsc4py.PETSc.KSP().create()
    ksp.setOperators(A)
    ksp.setType("preonly")
    ksp.getPC().setType("lu")
    if package is not None:
        ksp.getPC().setFactorSolverType(package)
    ksp.setFromOptions()
    ksp.solve(b, solution.vector)
    solution.vector.ghostUpdate(addv=petsc4py.PETSc.InsertMode.INSERT, mode=petsc4py.PETSc.ScatterMode.FORWARD)
    assert np.allclose(solution.vector.getArray(), np.arange(1, dim_V + 1))
    with dolfinx.io.VTXWriter(mesh.comm, "solution.bp", solution) as vtx_file:
        vtx_file.write(package_number * 1.0)
    ksp.destroy()

In [16]:
k = ufl.inner(ufl.grad(u), ufl.grad(v)) * dx
k_cpp = dolfinx.fem.form(k)

In [17]:
K = dolfinx.fem.petsc.assemble_matrix(k_cpp)
K.assemble()

In [18]:
expected = (0, 10.8, 54, 108)

In [19]:
import slepc4py
import slepc4py.SLEPc

In [20]:
for package in (None, "mumps", "superlu", "superlu_dist"):
    eps = slepc4py.SLEPc.EPS().create()
    eps.setOperators(K, A)
    eps.setProblemType(slepc4py.SLEPc.EPS.ProblemType.GHEP)
    eps.setWhichEigenpairs(slepc4py.SLEPc.EPS.Which.TARGET_REAL)
    eps.setTarget(1)
    st = eps.getST()
    st.setType(slepc4py.SLEPc.ST.Type.SINVERT)
    st.setShift(1)
    ksp = st.getKSP()
    ksp.setType("preonly")
    pc = ksp.getPC()
    pc.setType("lu")
    if package is not None:
        pc.setFactorSolverType(package)
    eps.solve()
    assert eps.getConverged() == len(expected)
    for (i, eig_i_ex) in enumerate(expected):
        eig_i = eps.getEigenvalue(i)
        assert np.isclose(eig_i.real, eig_i_ex)
        assert np.isclose(eig_i.imag, 0)
    eps.destroy()


In [21]:
%%bash

if [[ -n "$LD_PRELOAD" ]]; then
    export LD_PRELOAD=""
    ERROR_LIBRARIES=($(find /root/.cache/fenics -name '*\.so' -exec \
        bash -c 'ldd $0 | grep libstdc++.so.6 1>/dev/null 2>/dev/null && echo $0' {} \;))
    if [ ${#ERROR_LIBRARIES[@]} -eq 0 ]; then
        echo "No reference to libstdc++.so was found"
    else
        for ERROR_LIBRARY in "${ERROR_LIBRARIES[@]}"; do
            echo "Error: library $ERROR_LIBRARY depends on libstdc++.so"
            ldd -v $ERROR_LIBRARY
        done
        false
    fi
fi